In [146]:
from contextlib import redirect_stdout
from functools import wraps, partial
import os
from pathlib import Path

from htools import debug, LoggerMixin, delegate

In [104]:
def log_stdout(func=None, fname=''):
    """Decorator that logs all stdout produced by a function.
    
    Parameters
    ----------
    func: function
        If the decorator is used without parenthesis, the function will be
        passed in as the first argument. You never need to explicitly specify
        a function.
    fname: str
        Path to log file which will be created. If None is specified, the 
        default is to write to ./logs/wrapped_func_name.log. If specified, 
        this must be a keyword argument.
        
    Examples
    --------
    @log_stdout
    def foo(a, b=3):
        print(a)
        a *= b
        print(a)
        return a**b
        
    @log_stdout(fname='../data/mylog.log')
    def foo(a, b=3):
        ...
    """
    if not func: 
        return partial(log_stdout, fname=Path(fname))
    if not fname:
        fname = Path(f'./logs/{func.__name__}.log')

    @wraps(func)
    def wrapper(*args, **kwargs):
        os.makedirs(fname.parent, exist_ok=True)
        with open(fname, 'w') as f:
            with redirect_stdout(f):
                out = func(*args, **kwargs)
        return out
    return wrapper

In [94]:
@debug
def foo(a, b=3, c=(4,5), d=True):
    for i in range(a):
        if i % 2 == 0:
            print(i)
        elif i % 5 == 0:
            print('divis by 5')
        elif i > 50:
            print('*'*i)
    return tuple(b*i for i in c)

In [95]:
foo(24, 8)


CALLING foo(a=24, b=8, c=(4, 5), d=True)
0
2
4
divis by 5
6
8
10
12
14
divis by 5
16
18
20
22


(32, 40)

In [96]:
@log_stdout
@debug
def foo(a, b=3, c=(4,5), d=True):
    print(a, b, c, d)
    for i in range(a):
        if i % 2 == 0:
            print(i)
        elif i % 5 == 0:
            print('divis by 5')
        elif i > 50:
            print('*'*i)
    return tuple(b*i for i in c)

In [98]:
foo(22, 2)

(8, 10)

In [99]:
cat logs/foo.log


CALLING foo(a=22, b=2, c=(4, 5), d=True)
22 2 (4, 5) True
0
2
4
divis by 5
6
8
10
12
14
divis by 5
16
18
20


In [100]:
@log_stdout(fname='log2/mylog.log')
@debug
def foo(a, b=3, c=(4,5), d=True):
    print(a, b, c, d)
    for i in range(a):
        if i % 2 == 0:
            print(i)
        elif i % 5 == 0:
            print('divis by 5')
        elif i > 50:
            print('*'*i)
    return tuple(b*i for i in c)

In [101]:
foo(31, 19)

(76, 95)

In [102]:
cat log2/mylog.log  


CALLING foo(a=31, b=19, c=(4, 5), d=True)
31 19 (4, 5) True
0
2
4
divis by 5
6
8
10
12
14
divis by 5
16
18
20
22
24
divis by 5
26
28
30


## MultiOutput

TODO:
- check: 'write' mode doesn't seem to work [**answer:** it does, just not how I thought. See updated docstring for LoggerMixin]
- check: is there a way to make `logger` var accessible inside func? E.g. inject local var into a func via a decorator? Simply declaring nonlocal in wrapper did not work. [**answer:** I think I could do this but not simply. E.g. we can create a new signature with the added variable and supply a default value, but we can't just rely on nonlocal/global namespaces in the decorator to give the function access.]
- port MultiLogger and verbose_log() to htools

In [38]:
import numpy as np
import time

In [3]:
!ls ..

LICENSE.txt      data             htools.egg-info  setup.py
MANIFEST.in      dist             notebooks        tags
README.md        docs             requirements.txt
__pycache__      htools           setup.cfg


In [147]:
@delegate('logger')
class MultiLogger(LoggerMixin):
    
    def __init__(self, path, fmode='w', fmt='%(message)s'):
        self.logger = self.get_logger(path, fmode, 'info', fmt)
        
    def write(self, buf):
        if buf != '\n':
            self.logger.info(buf)

In [163]:
ml = MultiLogger('../data/ml.log')

In [164]:
with redirect_stdout(ml):
    for i in range(10):
        print(i)

0
1
2
3
4
5
6
7
8
9


In [165]:
ml.write('abcdefg')

abcdefg


In [166]:
!cat ../data/ml.log

0
1
2
3
4
5
6
7
8
9
abcdefg


In [167]:
ml.info('testing 3')

testing 3


In [168]:
!cat ../data/ml.log

0
1
2
3
4
5
6
7
8
9
abcdefg
testing 3


In [169]:
ml.write('writing 3')

writing 3


In [170]:
!cat ../data/ml.log

0
1
2
3
4
5
6
7
8
9
abcdefg
testing 3
writing 3


In [152]:
def verbose_log(path, fmode='w', fmt='%(message)s'):
    def decorator(func):
        logger = MultiLogger(path, fmode, fmt)
        @wraps(func)
        def wrapper(*args, **kwargs):
            with redirect_stdout(logger):
                return func(*args, **kwargs)
        return wrapper
    return decorator

In [153]:
@verbose_log('../data/decorated.log', 'w')
def main(a, b=77, *args, c=True, **kwargs):
    print('CALLING MAIN')
    print(a, b, c)
    print(args)
    print(kwargs)
    for i in range(5):
        wait = np.random.uniform()
        time.sleep(wait)
        print(i)

In [154]:
!cat ../data/decorated.log

In [155]:
main(7, c=True)

CALLING MAIN
7
 
77
 
True
()
{}
0
1
2
3
4


In [156]:
!cat ../data/decorated.log

CALLING MAIN
7
 
77
 
True
()
{}
0
1
2
3
4


In [157]:
main(0, c=False)

CALLING MAIN
0
 
77
 
False
()
{}
0
1
2
3
4


In [158]:
!cat ../data/decorated.log

CALLING MAIN
7
 
77
 
True
()
{}
0
1
2
3
4
CALLING MAIN
0
 
77
 
False
()
{}
0
1
2
3
4


## Scratch logger

In [135]:
fname = '../data/decorated.log'
logger = LoggerMixin().get_logger(fname, fmode='w')
logger

<Logger LoggerMixin (INFO)>

In [136]:
logger.info('testing')

2020-06-07 21:17:44,753 [INFO]: testing


In [137]:
!cat {fname}

2020-06-07 21:17:44,753 [INFO]: testing


In [138]:
logger.info('round 2')

2020-06-07 21:17:45,213 [INFO]: round 2


In [139]:
!cat {fname}

2020-06-07 21:17:44,753 [INFO]: testing
2020-06-07 21:17:45,213 [INFO]: round 2


In [140]:
logger = LoggerMixin().get_logger(fname, fmode='a')
logger

<Logger LoggerMixin (INFO)>

In [141]:
!cat {fname}

2020-06-07 21:17:44,753 [INFO]: testing
2020-06-07 21:17:45,213 [INFO]: round 2


In [142]:
logger.info('round 3')
logger.info('round 4')

2020-06-07 21:17:48,849 [INFO]: round 3
2020-06-07 21:17:48,852 [INFO]: round 4


In [143]:
!cat {fname}

2020-06-07 21:17:44,753 [INFO]: testing
2020-06-07 21:17:45,213 [INFO]: round 2
2020-06-07 21:17:48,849 [INFO]: round 3
2020-06-07 21:17:48,852 [INFO]: round 4


In [144]:
logger.info('round 5')
logger.info('round 6')

2020-06-07 21:17:53,396 [INFO]: round 5
2020-06-07 21:17:53,398 [INFO]: round 6


In [145]:
!cat {fname}

2020-06-07 21:17:44,753 [INFO]: testing
2020-06-07 21:17:45,213 [INFO]: round 2
2020-06-07 21:17:48,849 [INFO]: round 3
2020-06-07 21:17:48,852 [INFO]: round 4
2020-06-07 21:17:53,396 [INFO]: round 5
2020-06-07 21:17:53,398 [INFO]: round 6
